In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("~/Desktop/NCSA_genomics/prediabetic.csv")
data['DisplayTime'] = pd.to_datetime(data['DisplayTime'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [3]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['DisplayTime'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [4]:
def subSample(data):
    data = data.reset_index(drop=True)
    data['DisplayTime'] = pd.to_datetime(data['DisplayTime'])
    data['time_gap'] = data['DisplayTime'].shift(1)-data['DisplayTime'][0]
    data['time_gap'][0] = '00:00:00'
    mods = [0,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899]
    subset = pd.DataFrame()
    for i in range(1,len(data.index)):
        seconds = data['time_gap'][i].total_seconds()
        if (seconds%900) in mods:
            subj_id = data['subjectId'][i]
            gv = data['GlucoseValue'][i]
            dt = data['DisplayTime'][i]
            temp_df = pd.DataFrame({'Display Time':[dt], 'GlucoseValue':[gv], 'subjectId':[subj_id]})
            subset = pd.concat([temp_df,subset],ignore_index=True)
    subset = subset.iloc[::-1]
    subset = subset.reset_index(drop=True)
    data.drop(['time_gap'], axis=1, inplace=True)
    return subset

In [5]:
def modd(data):
    data['DisplayTime'] = data['DisplayTime'].dt.round('5min') 
    
    times = []
    for i in range(len(data.index)):
        times.append(data['DisplayTime'][i].time())
    data['Time'] = times  


    Modd = [] 
    s = 0
    gvDiff = 0

    for Time, df in data.groupby('Time'):
        gvDiff = df['GlucoseValue'] - df['GlucoseValue'].shift(-1)
        s = round(gvDiff.sum(),3)
        Modd.append(s)
    return round(mean(Modd),3)
    

In [6]:
# for subjectId, df in data.groupby('subjectId'):
#     df = fullDay(df)
#     df = subSample(df)
#     modd_value = modd(df)
#     print(subjectId, modd_value)

In [7]:
xx = data[data['subjectId'] == "1636-69-032"]
# xx = fullDay(xx)
# xx = subSample(xx)
print(modd(xx))

4.42


In [8]:
xx['DisplayTime'] = xx['DisplayTime'].dt.round('5min') 
dates = []
times = []
for i in range(len(xx.index)):
    dates.append(xx['DisplayTime'][i].date())
    times.append(xx['DisplayTime'][i].time())
xx['Date'] = dates   
xx['Time'] = times
xx

,Unnamed: 0,subjectId,DisplayTime,GlucoseValue,Time,Date
0,1,1636-69-032,2016-01-14 00:05:00,120,00:05:00,2016-01-14
1,2,1636-69-032,2016-01-14 00:10:00,120,00:10:00,2016-01-14
2,3,1636-69-032,2016-01-14 00:15:00,123,00:15:00,2016-01-14
3,4,1636-69-032,2016-01-14 00:20:00,126,00:20:00,2016-01-14
4,5,1636-69-032,2016-01-14 00:25:00,127,00:25:00,2016-01-14
...,...,...,...,...,...,...
1434,1435,1636-69-032,2016-01-18 23:35:00,136,23:35:00,2016-01-18
1435,1436,1636-69-032,2016-01-18 23:40:00,126,23:40:00,2016-01-18
1436,1437,1636-69-032,2016-01-18 23:45:00,132,23:45:00,2016-01-18
1437,1438,1636-69-032,2016-01-18 23:50:00,135,23:50:00,2016-01-18


In [9]:
day = xx['Date'].iloc[-1]-xx['Date'].iloc[0]
day = day.days
day

4

In [28]:
Modd = []
s = []
gvDiff = 0
for Time, df in xx.groupby('Time'):
    gvDiff = abs(df['GlucoseValue'] - df['GlucoseValue'].shift(-1))
    gvDiff = gvDiff.dropna()
    s.append(mean(gvDiff))
#     s = (gvDiff.sum())
#     Modd.append(s)
# print(mean(Modd))



In [24]:
mean(gvDiff)

11.0

In [30]:
mean(s)

16.223206018518518